In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('Dataset .csv')

In [3]:
data['Cuisines'] = data['Cuisines'].fillna('Unknown')

In [4]:
data['Features'] = (
    data['Cuisines'] + ' ' +
    data['Price range'].astype(str) + ' ' +
    data['Aggregate rating'].astype(str)
)

In [5]:
data['Features']

0             French, Japanese, Desserts 3 4.8
1                               Japanese 3 4.5
2       Seafood, Asian, Filipino, Indian 4 4.4
3                        Japanese, Sushi 4 4.9
4                       Japanese, Korean 4 4.8
                         ...                  
9546                             Turkish 3 4.1
9547     World Cuisine, Patisserie, Cafe 3 4.2
9548              Italian, World Cuisine 4 3.7
9549                     Restaurant Cafe 4 4.0
9550                                Cafe 2 4.0
Name: Features, Length: 9551, dtype: object

In [6]:
vectorizer = CountVectorizer()
feature_matrix = vectorizer.fit_transform(data['Features'])

In [7]:
feature_matrix

<9551x151 sparse matrix of type '<class 'numpy.int64'>'
	with 27081 stored elements in Compressed Sparse Row format>

In [8]:
similarity_matrix = cosine_similarity(feature_matrix)

In [9]:
similarity_matrix

array([[1.        , 0.57735027, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.57735027, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.70710678],
       [0.        , 0.        , 0.        , ..., 0.        , 0.70710678,
        1.        ]])

In [10]:
def recommend_restaurants(user_cuisine, user_price_range, user_rating, top_n=5):
    
    user_input = f"{user_cuisine} {user_price_range} {user_rating}"
    user_vector = vectorizer.transform([user_input])
    
    # Calculate similarity scores
    similarity_scores = cosine_similarity(user_vector, feature_matrix).flatten()
    
    # Sort and filter recommendations
    recommendations = data.copy()
    recommendations['Similarity'] = similarity_scores
    recommendations = recommendations[
        (recommendations['Aggregate rating'] >= user_rating) &
        (recommendations['Price range'] == user_price_range)
    ]
    recommendations = recommendations.sort_values(by='Similarity', ascending=False)
    
    return recommendations[['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating']].head(top_n)


In [11]:
user_preferences = {
    'user_cuisine': 'Japanese',
    'user_price_range': 3,
    'user_rating': 4.5
}
recommendations = recommend_restaurants(**user_preferences, top_n=5)
print(recommendations)

       Restaurant Name                  Cuisines  Price range  \
1     Izakaya Kikufuji                  Japanese            3   
9383              Roka           Japanese, Sushi            3   
9396      Bone Daddies           Ramen, Japanese            3   
9287        Sushi Masa           Sushi, Japanese            3   
1222       Yum Yum Cha  Chinese, Japanese, Sushi            3   

      Aggregate rating  
1                  4.5  
9383               4.6  
9396               4.6  
9287               4.9  
1222               4.5  


In [17]:
%%writefile app.py
import streamlit as st
import pandas as pd
st.title("Restaurant Recommendation System")

st.sidebar.header("Enter Your Preferences")
user_cuisine = st.sidebar.text_input("Cuisine (e.g., Japanese, Italian):", "Japanese")
user_price_range = st.sidebar.slider("Price Range (1-4):", 1, 4, 3)
user_rating = st.sidebar.slider("Minimum Rating (0.0 - 5.0):", 0.0, 5.0, 4.5)

if st.sidebar.button("Recommend"):
    recommendations = recommendations(
        user_cuisine=user_cuisine,
        user_price_range=user_price_range,
        user_rating=user_rating,
        top_n=5
    )
    if recommendations.empty:
        st.warning("No matching restaurants found. Try changing your preferences.")
    else:
        st.subheader("Top Recommendations")
        st.table(recommendations)



Overwriting app.py


In [18]:
!streamlit run app.py

^C
